<a href="https://colab.research.google.com/github/abdirimoof/ML/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [7]:
torch.cuda.is_available()

True

In [8]:
from torch.cuda import device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [9]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [10]:
# MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms)

100%|██████████| 9.91M/9.91M [00:00<00:00, 20.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 505kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.56MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.95MB/s]


In [11]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

Modelni yaratamiz

In [12]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.net = nn.Sequential(
            # 2block with Covn, Relu, Maxpool
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Flatten, Linear, ReLU, Linear
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
      return self.net(x)

In [13]:
model = SimpleCNN().to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
epoxa = 10

for epoch in range(epoxa):
  model.train()
  total_loss = 0.0

  for data, target in train_loader:
      data, target = data.to(device), target.to(device)

      #Forward - Oldga yurish
      output = model(data)
      loss = criterion(output, target)

      #Backward - Yangilash
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      total_loss += loss.item()
  print(f"Epoxa : {epoxa + 1}, O`rtacha loss: {total_loss/len(train_loader)}")


Epoxa : 11, O`rtacha loss: 0.008762816735699999
Epoxa : 11, O`rtacha loss: 0.007663540387036493
Epoxa : 11, O`rtacha loss: 0.006505507470030046
Epoxa : 11, O`rtacha loss: 0.005841921759582707
Epoxa : 11, O`rtacha loss: 0.006458925922775701
Epoxa : 11, O`rtacha loss: 0.004692235329257998
Epoxa : 11, O`rtacha loss: 0.003927411226620472
Epoxa : 11, O`rtacha loss: 0.005170951338341586
Epoxa : 11, O`rtacha loss: 0.0040739495597312166
Epoxa : 11, O`rtacha loss: 0.004427099650973845


In [20]:
  model.eval()

  correct = 0
  total = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)

      _, predicted = torch.max(output.data, 1)
      total += target.size(0)
      correct += (predicted == target).sum().item()

  print(f"Test accuracy: {100 * correct / total}%")

Test accuracy: 99.04%
